Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference Bert Model for High Performance with ONNX Runtime on AzureML #

This tutorial takes a pre-trained BERT model, converts it to ONNX, and deploys the ONNX model with ONNX Runtime through AzureML.
In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

## Contents

**Prerequisites** to set up your Azure ML work environments

**Obtain model and convert to ONNX**

**Deploy Bert model using ONNX Runtime and AzureML**

## Prerequisites

To run on AzureML, you need:
* Azure subscription
* Azure Machine Learning Workspace
* the Azure Machine Learning SDK
* the Azure CLI and the Azure Machine learning CLI extension (> version 2.2.2)

You might also find the following resources useful:
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* The [Azure Portal](https://portal.azure.com) allows you to track the status of your deployments.

In [1]:
# To install dependencies directly run the following
!pip install torch
!pip install transformers
!pip install azureml azureml.core
!pip install onnxruntime
!pip install matplotlib

# To create a a Jupter kernel from your conda environment, run the following. replacing <kernel name> with your own name
#   conda install -c anaconda ipykernel
#   python -m ipykernel install --user --name=<kernel name>

  Using cached azureml_core-1.40.0-py3-none-any.whl (2.7 MB)
ERROR: azureml-widgets 1.34.0 has requirement azureml-core~=1.34.0, but you'll have azureml-core 1.40.0 which is incompatible.
ERROR: azureml-train-core 1.34.0 has requirement azureml-core~=1.34.0, but you'll have azureml-core 1.40.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.34.0.post1 has requirement azureml-core~=1.34.0, but you'll have azureml-core 1.40.0 which is incompatible.
ERROR: azureml-train-automl-client 1.34.0 has requirement azureml-core~=1.34.0, but you'll have azureml-core 1.40.0 which is incompatible.
ERROR: azureml-tensorboard 1.34.0 has requirement azureml-core~=1.34.0, but you'll have azureml-core 1.40.0 which is incompatible.
ERROR: azureml-telemetry 1.34.0 has requirement azureml-core~=1.34.0, but you'll have azureml-core 1.40.0 which is incompatible.
ERROR: azureml-sdk 1.34.0 has requirement azureml-core~=1.34.0, but you'll have azureml-core 1.40.0 which is incompatible.
ERROR: azureml

## Obtain and convert PyTorch model to ONNX format

In the code below, we obtain a BERT model fine-tuned for question answering with the SQUAD dataset from HuggingFace.

If you'd like to pre-train a BERT model from scratch, follow the instructions in
[Pretraining of the BERT model](https://github.com/microsoft/AzureML-BERT/blob/master/pretrain/PyTorch/notebooks/BERT_Pretrain.ipynb). 
And if you would like to fine-tune the model with your own dataset, refer to  [AzureML Bert Eval Squad](https://github.com/microsoft/AzureML-BERT/blob/master/finetune/PyTorch/notebooks/BERT_Eval_SQUAD.ipynb)
or [AzureML Bert Eval GLUE](https://github.com/microsoft/AzureML-BERT/blob/master/finetune/PyTorch/notebooks/BERT_Eval_GLUE.ipynb).


### Define the tokenizer and model

In [5]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model_path = "./" + model_name + ".onnx"

from transformers import BertTokenizer, BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained(model_name)

In [7]:
max_seq_len = 1024

### Sample input and question

In [8]:
#question = "What is a major importance of Southern California in relation to California and the United States?"
#context = "Southern California, often abbreviated SoCal, is a geographic and cultural region that generally comprises California's southernmost 10 counties. The region is traditionally described as \"eight counties\", based on demographics and economic ties: Imperial, Los Angeles, Orange, Riverside, San Bernardino, San Diego, Santa Barbara, and Ventura. The more extensive 10-county definition, including Kern and San Luis Obispo counties, is also used based on historical political divisions. Southern California is a major economic center for the state of California and the United States."
question = "What is my name"
context = "My name is Natalie and my friend's name is Jane"

### Export the model

In [7]:
import torch

model_path = "./" + model_name + ".onnx"

# set the model to inference mode
# It is important to call torch_model.eval() or torch_model.train(False) before exporting the model, 
# to turn the model to inference mode. This is required since operators like dropout or batchnorm 
# behave differently in inference and training mode.
model.eval()

# Generate dummy inputs to the model. Adjust if neccessary
inputs = {
        'input_ids':   torch.randint(32, [1, 32], dtype=torch.long), # list of numerical ids for the tokenized text
        'attention_mask': torch.ones([1, 32], dtype=torch.long),     # dummy list of ones
        'token_type_ids':  torch.ones([1, 32], dtype=torch.long)     # dummy list of ones
    }

symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
torch.onnx.export(model,                                         # model being run
                  (inputs['input_ids'],
                   inputs['attention_mask'], 
                   inputs['token_type_ids']),                    # model input (or a tuple for multiple inputs)
                  model_path,                                    # where to save the model (can be a file or file-like object)
                  opset_version=11,                              # the ONNX version to export the model to
                  do_constant_folding=True,                      # whether to execute constant folding for optimization
                  input_names=['input_ids',
                               'input_mask', 
                               'segment_ids'],                   # the model's input names
                  output_names=['start_logits', "end_logits"],   # the model's output names
                  dynamic_axes={'input_ids': symbolic_names,
                                'input_mask' : symbolic_names,
                                'segment_ids' : symbolic_names,
                                'start_logits' : symbolic_names, 
                                'end_logits': symbolic_names})   # variable length axes

In [4]:
%run export.py

NameError: name 'model_name' is not defined

## Run the ONNX model with ONNX Runtime



The following code runs the ONNX model with ONNX Runtime. You can test it locally before deploying it to Azure Machine Learning.

The `init()` function is called at startup, performing the one-off operations such as creating the tokenizer and the ONNX Runtime session.

The `run()` function is called when we run the model using the Azure ML web service.
Add neccessary `preprocess()` and `postprocess()` steps.

For local testing and comparison, we also run the PyTorch model.


In [11]:
%%writefile score.py
import os
import logging
import json
import numpy as np
import onnxruntime
import transformers
import torch


def preprocess(question, context):
    print("Question:", question)
    print("Context: ", context)
    encoded_input = tokenizer(question, context)
    tokens = tokenizer.convert_ids_to_tokens(encoded_input.input_ids)
    print(tokens)
    return (encoded_input.input_ids, encoded_input.attention_mask, encoded_input.token_type_ids, tokens)

def postprocess(tokens, start, end):
    print("Start:", start)
    print("End:", end)
    results = {}
    answer_start = np.argmax(start)
    answer_end = np.argmax(end)
    print("Start: ", answer_start)
    print("End: ", answer_end)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
        results['answer'] = answer.capitalize()
    else:
        results['error'] = "I am unable to find the answer to this question. Can you please ask another question?"
    return results

def init():
    global tokenizer, session, model

    model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
    model = transformers.BertForQuestionAnswering.from_pretrained(model_name)

    # use AZUREML_MODEL_DIR to get your deployed model(s). If multiple models are deployed, 
    # model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), '$MODEL_NAME/$VERSION/$MODEL_FILE_NAME')
    model_dir = os.getenv('AZUREML_MODEL_DIR')
    if model_dir == None:
        model_dir = "./"
    model_path = os.path.join(model_dir, model_name + ".onnx")

    # Create the tokenizer
    tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

    # Create an ONNX Runtime session to run the ONNX model
    session = onnxruntime.InferenceSession(model_path, providers=["CPUExecutionProvider"])  


def run_pytorch(raw_data):
    inputs = json.loads(raw_data)

    model.eval()

    logging.info("Question:", inputs["question"])
    logging.info("Context: ", inputs["context"])

    input_ids, input_mask, segment_ids, tokens = preprocess(inputs["question"], inputs["context"])

    print(input_ids)

    model_outputs = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

    outputs = postprocess(tokens, model_outputs.start_logits.detach().numpy(), model_outputs.end_logits.detach().numpy())

    print(outputs)


def run(raw_data):
    logging.info("Request received")
    inputs = json.loads(raw_data)

    logging.info(inputs)

    # Preprocess the question and context into tokenized ids
    input_ids, input_mask, segment_ids, tokens = preprocess(inputs["question"], inputs["context"])

    print(input_ids)
    
    # Format the inputs for ONNX Runtime
    model_inputs = {
        'input_ids':   [input_ids], 
        'input_mask':  [input_mask],
        'segment_ids': [segment_ids]
        }
                  
    outputs = session.run(['start_logits', 'end_logits'], model_inputs)
    
    # Post process the output of the model into an answer (or an error if the question could not be answered)
    return postprocess(tokens, outputs[0], outputs[1])


if __name__ == '__main__':
    init()

    #input = "{\"question\": \"What is my name?\", \"context\": \"My name is Natalie, and my sister's name is also Nathalie and my brother's name is dufas and my friend's name is Remy\"}"

    #input = "{\"question\": \"What is Dolly Parton's middle name?\", \"context\": \"Dolly Rebecca Parton (born January 19, 1946) is an American singer-songwriter, actress, and businesswoman, known primarily for her work in country music. After achieving success as a songwriter for others, Parton made her album debut in 1967 with Hello, I'm Dolly, which led to success during the remainder of the 1960s (both as a solo artist and with a series of duet albums with Porter Wagoner), before her sales and chart peak came during the 1970s and continued into the 1980s. Parton's albums in the 1990s did not sell as well, but she achieved commercial success again in the new millennium and has released albums on various independent labels since 2000, including her own label, Dolly Records. She has sold more than 100 million records worldwide.\"}"

    input = "{\"question\": \"What is Dolly Parton's middle name?\", \"context\": \"Dolly Rebecca Parton is an American singer-songwriter\"}"

    run_pytorch(input)
    print(run(input))



Overwriting score.py


In [12]:
%run score.py

Question: What is Dolly Parton's middle name?
Context:  Dolly Rebecca Parton is an American singer-songwriter
['[CLS]', 'what', 'is', 'dolly', 'part', '##on', "'", 's', 'middle', 'name', '?', '[SEP]', 'dolly', 'rebecca', 'part', '##on', 'is', 'an', 'american', 'singer', '-', 'songwriter', '[SEP]']
[101, 2054, 2003, 19958, 2112, 2239, 1005, 1055, 2690, 2171, 1029, 102, 19958, 9423, 2112, 2239, 2003, 2019, 2137, 3220, 1011, 6009, 102]
Start: [[-5.7373166 -2.0882578 -6.6572905 -4.736242  -7.208626  -8.217027
  -8.110763  -7.1893578 -5.3911524 -6.7148128 -8.595815  -5.7372403
   2.8533413  6.8337164 -4.656281  -5.8789706 -5.630715  -5.2006907
  -4.8701143 -4.541009  -7.904859  -5.5882034 -5.737352 ]]
End: [[-1.1449381  -0.75719965 -4.437096   -4.4248323  -6.008774   -2.813389
  -5.418213   -4.610936   -4.350332   -4.110813   -5.120949   -1.144799
   0.30320665  7.0895014  -3.892273    2.120804   -4.046403   -4.8648496
  -3.4395008  -4.4262953  -5.8198347  -2.6823187  -1.144946  ]]
Start:  

## Deploy model with ONNX Runtime through AzureML

Now that we have the ONNX model and the code to run it with ONNX Runtime, we can deploy it using Azure ML.



## Check your environment

In [12]:
# Check core SDK version number
import azureml.core
import onnxruntime
import torch
import transformers

print("Transformers version: ", transformers.__version__)
torch_version = torch.__version__
print("Torch (ONNX exporter) version: ", torch_version)
print("Azure SDK version:", azureml.core.VERSION)
print("ONNX Runtime version: ", onnxruntime.__version__)


Transformers version:  4.17.0
Torch (ONNX exporter) version:  1.10.0
Azure SDK version: 1.40.0
ONNX Runtime version:  1.11.0


### Load your Azure ML workspace

We begin by instantiating a workspace object from the existing workspace created earlier in the configuration notebook.

In [13]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = '\n')

ort_training_dev
australiaeast
onnx_training


## Register your model with Azure ML

Now we upload the model and register it in the workspace.


In [14]:
from azureml.core.model import Model

model = Model.register(model_path = model_path,                 # Name of the registered model in your workspace.
                       model_name = model_name,            # Local ONNX model to upload and register as a model
                       model_framework=Model.Framework.ONNX ,   # Framework used to create the model.
                       model_framework_version=torch_version,   # Version of ONNX used to create the model.
                       tags = {"onnx": "demo"},
                       description = "HuggingFace Bert model fine-tuned with SQuAd and exported from PyTorch",
                       workspace = ws)

./bert-large-uncased-whole-word-masking-finetuned-squad.onnx
bert-large-uncased-whole-word-masking-finetuned-squad
Registering model bert-large-uncased-whole-word-masking-finetuned-squad


#### Displaying your registered models

You can list out all the models that you have registered in this workspace.

In [15]:
models = ws.models
for name, m in models.items():
    print("Name:", name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)
    
#     # If you'd like to delete the models from workspace
#     model_to_delete = Model(ws, name)
#     model_to_delete.delete()

Name: hf-gpt2.onnx 	Version: 1 	Description: ONNX version of base HuggingFace GPT-2 {}
Name: hf-gpt2.pt 	Version: 1 	Description: GPT-2 model saved from pre-trained HuggingFace {}
Name: pytorch-hf-gpt-onnx-int8 	Version: 1 	Description: None {}
Name: pytorch-hf-gpt2-wikitext103 	Version: 1 	Description: None {}
Name: pt-ort-hf-gpt2-wt103-full 	Version: 1 	Description: HuggingFace GPT-2 fine-tuned with PyTorch ORT using Wikitext103 {}
Name: sample-densenet-onnx-model 	Version: 1 	Description: None {}
Name: bert-large-uncased-whole-word-masking-finetuned-squad 	Version: 2 	Description: HuggingFace Bert model fine-tuned with SQuAd and exported from PyTorch {'onnx': 'demo'}


## Deploy the model 

We are now going to deploy our ONNX model on Azure ML using ONNX Runtime.

Firstly we will test the deployment using an Azure Container Instance, then deploy the model for production using an Azure ML endpoint.



## Deploy Model and Scoring code as an Azure ML endpoint

Note that the endpoint interface of the Python SDK is in private preview. For this section, we will use the Azure ML CLI.

There are three YML files in the `yml` folder:
* `env.yml`: A conda environment specification, from which the execution environment of the endpoint will be generated
* `endpoint.yml`: The endpoint specification, which simply contains the name of the endpoint and the authorization method
* `deployment.yml`: The deployment specification, which contains specifications of the scoring code, model, and environment. You can create multiple deployments per endpoint, and route different amounts of traffic to the deployments. For this example, we will create only one deployment.

In [ ]:
# Check how to specify the proportion of traffic
!az ml oneline-endpoint create --name question-answer-ort --file yml/endpoint.yml
!az ml online-deployment create --endpoint-name question-answer-ort --name blue --file yml/deployment.yml 

### Test the endpoint
az ml online-endpoint invoke --name question-answer-ort --request-file test-data.json

## Success!

If you've made it this far, you've deployed a working endpoint that answers a question using an ONNX model.

## Measure the inference latency

## Clean up Azure resources